# Introducción práctica a la ciencia de datos en R

**Wenceslao Arroyo-Machado**

Esta es una [R Markdown](http://rmarkdown.rstudio.com) Notebook con todos los contenidos del curso [Introducción práctica a la ciencia de datos en R](https://yosigopublicando.ugr.es/courses/introduccion-practica-a-la-ciencia-de-datos-en-r/) y su [repetición](https://yosigopublicando.ugr.es/courses/introduccion-practica-a-la-ciencia-de-datos-en-r-redux/). El objetivo es introducir los procesos básicos de ciencia de datos para el procesamiento de datos (lectura, procesamiento y exportación).

# Preparación

## Paquetes

Vamos a hacer uso de dos paquetes de R: `data.table()` y `dplyr()`. Puedes importarlos a la sesión mediante la función `library()`.

In [1]:
library(data.table)
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:data.table’:

    between, first, last

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



En caso de que no tengas alguno de los paquetes puedes instalarlos rápidamente con `install.packages()` y tras ello importarlos usando la función `library()`.

In [ ]:
#install.packages('data.table')
#install.packages('dplyr')

## Datos

Para esta sesión vamos a usar un subconjunto de datos de [IMDB](https://www.imdb.com/interfaces/). Se han recuperado varios datasets y filtrado previamente a películas, cortos y videojuegos de los años 90. Los datos están disponibles en el directorio *data*.

En concreto vamos a usar tres datasets:

-   **title_basics.tsv** (96026 x 8) - dataset principal con los datos básicos de las obras
    -   *tconst* - id de la obra
    -   *titleType* - tipo de obra
    -   *primaryTitle* - título principal
    -   *originalTitle* - título original
    -   *isAdult* - contenido adulto
    -   *startYear* - año
    -   *runtimeMinutes* - duración
    -   *genres* - género/s


-   **title_ratings.tsv** (44028 x 3) - puntuaciones promedio
    -   *tconst* - id de la obra
    -   *averageRating* - puntuación promedio
    -   *numVotes*- número de votos


-   **title_akas.tsv** (371351 x 8) - traducciones de los títulos
    -   *titleId* - id de la obra
    -   *ordering* - orden
    -   *title* - título
    -   *region* - región del título
    -   *language* - idioma
    -   *types* - atributos
    -   *attributes* - términos adicionales
    -   *isOriginalTitle* - título original

# Lectura de datos

Antes de importar los datos a RStudio o cualquier otra herramienta es necesario revisar los ficheros manualmente para conocer su estructura y características. Los formatos estándar son en texto plano delimitado por comas o punto y coma (*.csv*) y tabulador (*.tsv*). Es fundamental saber:

-   **Codificación** - juego de caracteres, generalmente usarás UTF-8.
-   **Delimitador** - carácter de separación, lo normal es la coma (,), punto y coma (;) y tabulador (\\t).
-   **Calificador** - carácter que evita que en cadenas de texto donde esta presente un caracter igual al delimitador no lo reconozca como tal, lo normal son las comillas dobles (") o comillas simples (').
-   **Caractereres nulos** - cadenas de caracteres que reconocer como valores nulos.

Estos son las opciones que más problemas te pueden dar. Sin embargo existen otras como el caracter usado para los decimales, el uso o no de cabecera...

## read.table()

Existe una función base que permite importar archivos en texto plano. La función es `read.table()`. Tanto esta como otras funciones vienen con parámetros por defecto, usa `?read.table()` para conocerlos. En este caso verás que muchos vienen desactivados.

**Ejemplo:** Vamos a importar el archivo con la información básica de películas.

In [2]:
df <- read.table(file = 'data/title_basics.tsv',
                 header = TRUE, # mi archivo tiene cabecera
                 sep = '\t', # el delimitador es el tabulador
                 quote = '', # no tengo calificadores
                 na.strings = 'null', # caracteres nulos
                 comment.char = '', # no tengo comentarios
                 encoding = 'UTF-8') # codificación

Desde el IDE de RStudio puedes comprobar cómo se han importado los datos e incluso visualizarlos. Podemos usar para ello `summary()` para obtener un resumen de las variables/instancias y `head()`. para visualizar las primeras líneas del *data.frame*. Los datos son importados como *data.frame*, el cual puede ser visto como una tabla o una hoja de cálculo. Los *data.frame* cuentan con diferentes variables o columnas, pudiendo ser cada una de un tipo (texto,, número, booleano...) y existen diferentes instancias o filas.

In [3]:
summary(df)
head(df)

       tconst          titleType                          primaryTitle  
 tt0015724:    1   movie    :44754   Requiem                    :   15  
 tt0038086:    1   short    :26208   The Gift                   :   15  
 tt0040844:    1   tvMovie  :17757   Angel                      :   11  
 tt0059325:    1   tvShort  : 1106   Maria                      :   10  
 tt0059900:    1   videoGame: 6201   Statsministerens nytårstale:   10  
 tt0065188:    1                     Payback                    :    9  
 (Other)  :96020                     (Other)                    :95956  
                     originalTitle      isAdult          startYear   
 Requiem                    :   15   Min.   :0.00000   Min.   :1990  
 The Gift                   :   14   1st Qu.:0.00000   1st Qu.:1992  
 Angel                      :   10   Median :0.00000   Median :1995  
 Maria                      :   10   Mean   :0.01504   Mean   :1995  
 Statsministerens nytårstale:   10   3rd Qu.:0.00000   3rd Qu.:199

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
tt0015724,movie,Dama de noche,Dama de noche,0,1993,102,"Drama,Mystery,Romance"
tt0038086,movie,Shiva und die Galgenblume,Shiva und die Galgenblume,0,1993,NA,Thriller
tt0040844,short,Crossroads of Laredo,Crossroads of Laredo,0,1995,30,"Short,Western"
tt0059325,movie,Born in '45,Jahrgang 45,0,1990,100,"Drama,Romance"
tt0059900,movie,"Wenn du groß bist, lieber Adam","Wenn du groß bist, lieber Adam",0,1990,78,"Drama,Fantasy"
tt0065188,movie,"Vojtech, receny sirotek","Vojtech, receny sirotek",0,1990,80,Drama


Existen diferente variantes de `read.table()` que tomando está como base incluyen algunos parámetros concretos para importar determinados archivos de forma ágil. Estas son:

-   `read.csv()` - archivos separados por comas y con puntos por decimales.
-   `read.csv2()` - archivos separados por comas y con comas por decimales.
-   `read.delim()` - archivos separados por tabuladores y con puntos por decimales.
-   `read.delim2()` - archivos separados por tabuladores y con comas por decimales.

**Ejemplo:** El archivo anterior se puede importar directamente con `read.delim()`.


In [4]:
df <- read.delim(file = 'data/title_basics.tsv',
                 quote = '', # no tengo calificadores
                 na.strings = 'null', # caracteres nulos
                 encoding = 'UTF-8') # codificación

Como está enmascarando la función `read.table()` podemos usar los mismos argumentos que esta admite.

Otro elemento a considerar en la importación de los datos es el tipo de dato. Este formato es asignado por defecto pero puede que en ocasiones queramos cambiarlo. Podemos hacerlo desde la propia importación de datos usando el argumento *colClasses*, indicando la variable y su tipo.

**Ejemplo:** Importar el archivo anterior pero con la columnas de años como texto en lugar de números.

In [5]:
df <- read.delim(file = 'data/title_basics.tsv',
                 quote = '', # no tengo calificadores
                 na.strings = 'null', # caracteres nulos
                 encoding = 'UTF-8', # codificación
                 colClasses = c('startYear'='character')) # forzamos que los años sean texto

En línea con ello, también podemos hacer que las variables de tipo texto sean reconocidas como factores con el argumento *stringsAsFactors*. Esta opción hasta hace unos años venía activada por defecto.

**Ejemplo:** Importar el archivo anterior pero con las variables de texto como factores.

In [6]:
df <- read.delim(file = 'data/title_basics.tsv',
                 quote = '', # no tengo calificadores
                 na.strings = 'null', # caracteres nulos
                 encoding = 'UTF-8', # codificación
                 stringsAsFactors = TRUE) # texto como factores

## fread()

Existen funciones en otros paquetes que facilitan aún más este trabajo, como es el caso de `fread()` del paquete `data.table`. Esta función es más rápida y directamente reconoce el delimitador, aunque es recomendable curarse en salud e indicarlo. A grandes rasgos, los argumentos de `read.table()` también están presentes aquí, por lo que es muy fácil trabajar con esta función.

**Ejemplo:** Importar el archivo anterior con la información básica de películas con `fread()`.

In [7]:
df <- fread(file = 'data/title_basics.tsv',
            quote = '', # no tengo calificadores
            na.strings = 'null', # caracteres nulos
            encoding = 'UTF-8') # codificación

Sin embargo, hay un aspecto a tener en cuenta. Esta función por defecto importa los datos como tipo *data.table* en lugar de *data.frame*. Este formato es más ágil y avanzado, pero puedes hacer que se importe como *data.frame* con el argumento *data.table*. En nuestro caso trabajaremos con *data.frame* por ser un curso introductorio.

In [9]:
df <- fread(file = 'data/title_basics.tsv',
            quote = '', # no tengo calificadores
            na.strings = 'null', # caracteres nulos
            encoding = 'UTF-8', # codificación
            data.table = FALSE) # data.table o data.frame

# Jugando con los datos

Una vez importados los datos podemos realizar diferentes procesos con ellos para filtrarlos, combinarlos... A lo largo de esta sección trabajaremos con las opciones base de R así como con el paquete `dplyr`.

## Consideraciones previas

-   Lo habitual al realizar alguna manipulación de datos es almacenar el *data.frame* resultante en una nueva variable o sustituir la actual. Si no estás muy segura/o del resultado de la operación o crees que más adelante tendrás que volver al conjunto original crea una variable nueva.

-   En el caso de `dplyr`, es posible realizar varias operaciones consecutivas de forma ágil. Este paquete habilita el operador pipe (%\>%) que facilita este proceso. Más adelante veremos su funcionamiento.

## Ordenar filas

Una primera operación sencilla y básica es ordenar filas en base a los valores de una columna. La forma básica es a través de la función `order()`. indicando la columna y con el argumento *decreasing* si queremos ordenar de manera ascendente o descendente. Si no lo indicamos por defecto lo hace ascendente.

**Ejemplo:** Ordenar el *data.frame* por *startYear* en orden descendente.

In [10]:
df_filtrado <- df[order(df$startYear, decreasing = TRUE),]
head(df_filtrado)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
17,tt0082473,short,The Great Barrier Reef,Great Barrier Reef,0,1999,39,"Documentary,Short"
4796,tt0104674,tvMovie,Lakota Moon,Lakota Moon,0,1999,85,Western
9423,tt0110445,movie,Making Waves,Making Waves,0,1999,105,"Comedy,Fantasy"
9987,tt0111068,movie,Sangharsh,Sangharsh,0,1999,127,"Action,Crime,Drama"
10300,tt0111427,movie,Tiden är en dröm,Tiden är en dröm,0,1999,125,"Documentary,History"
10897,tt0112444,movie,My Teacher's Wife,My Teacher's Wife,0,1999,89,"Comedy,Romance"


En el caso de `dplyr` podemos hacerlo con la función `arrange()`. En este caso si a continuación incluimos el nombre de la variable ordenamos de manera ascendente, pero si queremos de forma descendente tenemos incluir la variable dentro de la función `desc()`. Fíjate que las variables se indican directamente sin nada más, ni siquiera comillas, esto es algo que se va a repetir en este paquete.

**Ejemplo:** Ordenar el *data.frame* por *startYear* en orden descendente con `dplyr`.

In [11]:
df_filtrado <- arrange(df, desc(startYear))
head(df_filtrado)

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
tt0082473,short,The Great Barrier Reef,Great Barrier Reef,0,1999,39,"Documentary,Short"
tt0104674,tvMovie,Lakota Moon,Lakota Moon,0,1999,85,Western
tt0110445,movie,Making Waves,Making Waves,0,1999,105,"Comedy,Fantasy"
tt0111068,movie,Sangharsh,Sangharsh,0,1999,127,"Action,Crime,Drama"
tt0111427,movie,Tiden är en dröm,Tiden är en dröm,0,1999,125,"Documentary,History"
tt0112444,movie,My Teacher's Wife,My Teacher's Wife,0,1999,89,"Comedy,Romance"


## Selección de columnas

La selección de columnas puede hacerse indicando directamente el nombre de la/s columna/s que queremos seleccionar o el índice de su posición en el *data.frame*. Es útil además para cambiar el orden de las columnas.

**Ejemplo:** Seleccionar las variables *primaryTitle* y *titleType* (en este orden).

In [12]:
df_filtrado <- df[,c('primaryTitle', 'titleType')]
df_filtrado <- df[,c(3,2)] # igual que el anterior pero con el índice
head(df_filtrado)

primaryTitle,titleType
Dama de noche,movie
Shiva und die Galgenblume,movie
Crossroads of Laredo,short
Born in '45,movie
"Wenn du groß bist, lieber Adam",movie
"Vojtech, receny sirotek",movie


En el caso de `dplyr` podemos hacerlo con la función `select()`, indicando primero el *data.frame* a filtrar y tras ello el nombre de las diferentes variables.

**Ejemplo:** Seleccionar las variables *primaryTitle* y *titleType* (en este orden) con `dplyr`.

In [13]:
df_filtrado <- select(df, primaryTitle, titleType)
head(df_filtrado)

primaryTitle,titleType
Dama de noche,movie
Shiva und die Galgenblume,movie
Crossroads of Laredo,short
Born in '45,movie
"Wenn du groß bist, lieber Adam",movie
"Vojtech, receny sirotek",movie


## Selección de casos

### Posición

La selección de casos puede hacerse indicando directamente el nombre de la/s fila/s (si lo tiene) o, lo más habitual, el índice de su posición en el *data.frame*.

**Ejemplo:** Seleccionar las filas de la posición 100 a la 200.

In [14]:
df_filtrado <- df[c(100:200),]
head(df_filtrado)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
100,tt0096154,movie,Spies Inc.,Spies Inc.,0,1992,93,"Adventure,Comedy"
101,tt0096174,movie,Stela,Stela,0,1990,86,Drama
102,tt0096177,movie,Still Life: The Fine Art of Murder,Still Life,0,1990,83,"Mystery,Thriller"
103,tt0096222,movie,To Miss with Love,Tao xue wai zhuan,0,1992,90,"Comedy,Drama"
104,tt0096254,short,That Burning Question,That Burning Question,0,1990,30,Short
105,tt0096345,short,Unter Freunden,Unter Freunden,0,1990,15,Short


En el caso de `dplyr` podemos hacerlo con la función `slice()`, indicando primero el *data.frame* y después la posición.

**Ejemplo:** Seleccionar las filas de la posición 100 a la 200 con `dplyr`.df_filtrado <- slice(df, 100:200)
head(df_filtrado)

In [15]:
df_filtrado <- slice(df, 100:200)
head(df_filtrado)

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
tt0096154,movie,Spies Inc.,Spies Inc.,0,1992,93,"Adventure,Comedy"
tt0096174,movie,Stela,Stela,0,1990,86,Drama
tt0096177,movie,Still Life: The Fine Art of Murder,Still Life,0,1990,83,"Mystery,Thriller"
tt0096222,movie,To Miss with Love,Tao xue wai zhuan,0,1992,90,"Comedy,Drama"
tt0096254,short,That Burning Question,That Burning Question,0,1990,30,Short
tt0096345,short,Unter Freunden,Unter Freunden,0,1990,15,Short


### Filtrar (lógico)

Es posible en ambos casos hacer la selección filtrando mediante criterios. De base puede hacerse con la función `which()` estableciendo criterios lógicos.

Operadores lógicos comunes:

-   **\<** - menor que
-   \> - mayor que
-   \>= - mayor o igual que
-   \<= - menor o igual que
-   ! - negación
-   == - exactamente
-   != - distinto
-   %in% - pertenece a un grupo

**Ejemplo:** Seleccionar las filas con un valor de *startYear* mayor a 1995.


In [16]:
df_filtrado <- df[which(df$startYear > 1995),]
head(df_filtrado)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
17,tt0082473,short,The Great Barrier Reef,Great Barrier Reef,0,1999,39,"Documentary,Short"
47,tt0093061,movie,Funkytown,Funkytown,0,1998,82,"Documentary,Music"
86,tt0095469,movie,Ku jia tian xia,Ku jia tian xia,0,1998,102,Comedy
155,tt0097066,movie,Citizen Tania,Citizen Tania,0,1997,87,"Crime,Drama"
197,tt0097566,movie,Im Süden meiner Seele,Im Süden meiner Seele,0,1996,NA,NA
1023,tt0099840,movie,In the Shadow of the Sandcastle,In the Shadow of the Sandcastle,0,1996,79,"Crime,Drama"


**Ejemplo:** Seleccionar las filas con un valor de *startYear* entre 1995 y 1997.

In [18]:
df_filtrado <- df[which(df$startYear %in% c(1995:1997)),]
head(df_filtrado)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
3,tt0040844,short,Crossroads of Laredo,Crossroads of Laredo,0,1995,30,"Short,Western"
7,tt0067460,movie,My Kingdom for...,My Kingdom For...,0,1995,85,"Documentary,Sport"
18,tt0082631,tvMovie,Short Working Day,Krótki dzien pracy,0,1995,73,Drama
31,tt0089544,tvMovie,Man(n) sucht Frau,Man(n) sucht Frau,0,1995,85,Comedy
32,tt0090000,movie,Wet Sexes in the Sun,Sexos húmedos al sol,1,1995,NA,Adult
76,tt0095102,movie,Fallen Couple,Les ennemis de la mafia,0,1995,87,Thriller


En el caso de `dplyr` podemos hacerlo con la función `filter()`, indicando primero el *data.frame* y luego el criterio.

**Ejemplo:** Seleccionar las filas con un valor de *startYear* mayor a 1995 con `dplyr`.

In [19]:
df_filtrado <- filter(df, startYear > 1995)
head(df_filtrado)

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
tt0082473,short,The Great Barrier Reef,Great Barrier Reef,0,1999,39,"Documentary,Short"
tt0093061,movie,Funkytown,Funkytown,0,1998,82,"Documentary,Music"
tt0095469,movie,Ku jia tian xia,Ku jia tian xia,0,1998,102,Comedy
tt0097066,movie,Citizen Tania,Citizen Tania,0,1997,87,"Crime,Drama"
tt0097566,movie,Im Süden meiner Seele,Im Süden meiner Seele,0,1996,NA,NA
tt0099840,movie,In the Shadow of the Sandcastle,In the Shadow of the Sandcastle,0,1996,79,"Crime,Drama"


**Ejemplo:** Seleccionar las filas con un valor de *startYear* entre 1995 y 1997 con `dplyr`.

In [20]:
df_filtrado <- filter(df, startYear %in% c(1995:1997))
head(df_filtrado)

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
tt0040844,short,Crossroads of Laredo,Crossroads of Laredo,0,1995,30,"Short,Western"
tt0067460,movie,My Kingdom for...,My Kingdom For...,0,1995,85,"Documentary,Sport"
tt0082631,tvMovie,Short Working Day,Krótki dzien pracy,0,1995,73,Drama
tt0089544,tvMovie,Man(n) sucht Frau,Man(n) sucht Frau,0,1995,85,Comedy
tt0090000,movie,Wet Sexes in the Sun,Sexos húmedos al sol,1,1995,NA,Adult
tt0095102,movie,Fallen Couple,Les ennemis de la mafia,0,1995,87,Thriller


Asimismo, podemos combinar diferentes criterios con los operadores lógicos AND y OR. De base puede usarse AND con & y OR con \|.

**Ejemplo:** Seleccionar las filas con un valor de *startYear* mayor a 1995 y con *titleTyple* movie.

In [21]:
df_filtrado <- df[which(df$startYear > 1995 & df$titleType == 'movie'),]
head(df_filtrado)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
47,tt0093061,movie,Funkytown,Funkytown,0,1998,82,"Documentary,Music"
86,tt0095469,movie,Ku jia tian xia,Ku jia tian xia,0,1998,102,Comedy
155,tt0097066,movie,Citizen Tania,Citizen Tania,0,1997,87,"Crime,Drama"
197,tt0097566,movie,Im Süden meiner Seele,Im Süden meiner Seele,0,1996,NA,NA
1023,tt0099840,movie,In the Shadow of the Sandcastle,In the Shadow of the Sandcastle,0,1996,79,"Crime,Drama"
2875,tt0102267,movie,Lifebreath,Lifebreath,0,1997,90,"Drama,Romance,Thriller"


En `dplyr` podemos hacer uso de AND con & o separando los criterios con comas y de OR usando directamente \|.

**Ejemplo:** Seleccionar las filas con un valor de *startYear* mayor a 1995 y con *titleTyple* movie con `dplyr`.

In [22]:
df_filtrado <- filter(df, startYear > 1995, titleType == 'movie')
head(df_filtrado)

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
tt0093061,movie,Funkytown,Funkytown,0,1998,82,"Documentary,Music"
tt0095469,movie,Ku jia tian xia,Ku jia tian xia,0,1998,102,Comedy
tt0097066,movie,Citizen Tania,Citizen Tania,0,1997,87,"Crime,Drama"
tt0097566,movie,Im Süden meiner Seele,Im Süden meiner Seele,0,1996,NA,NA
tt0099840,movie,In the Shadow of the Sandcastle,In the Shadow of the Sandcastle,0,1996,79,"Crime,Drama"
tt0102267,movie,Lifebreath,Lifebreath,0,1997,90,"Drama,Romance,Thriller"


Un tipo de filtrado de mucha utilidad es a partir de valores nulos, para eliminar aquellas instancias que contienen en una o varias columnas valores nulos. Para ello, podemos usar la función `is.na()` junto al operador *!*.

**Ejemplo:** Seleccionar las filas que no tienen valores nulos en *genres*.

In [23]:
df_filtrado <- df[which(!is.na(df$genres)),]
head(df_filtrado)

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
tt0015724,movie,Dama de noche,Dama de noche,0,1993,102,"Drama,Mystery,Romance"
tt0038086,movie,Shiva und die Galgenblume,Shiva und die Galgenblume,0,1993,NA,Thriller
tt0040844,short,Crossroads of Laredo,Crossroads of Laredo,0,1995,30,"Short,Western"
tt0059325,movie,Born in '45,Jahrgang 45,0,1990,100,"Drama,Romance"
tt0059900,movie,"Wenn du groß bist, lieber Adam","Wenn du groß bist, lieber Adam",0,1990,78,"Drama,Fantasy"
tt0065188,movie,"Vojtech, receny sirotek","Vojtech, receny sirotek",0,1990,80,Drama


**Ejemplo:** Seleccionar las filas que no tienen valores nulos en *genres* con `dplyr`.

In [24]:
df_filtrado <- filter(df, !is.na(genres))
head(df_filtrado)

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
tt0015724,movie,Dama de noche,Dama de noche,0,1993,102,"Drama,Mystery,Romance"
tt0038086,movie,Shiva und die Galgenblume,Shiva und die Galgenblume,0,1993,NA,Thriller
tt0040844,short,Crossroads of Laredo,Crossroads of Laredo,0,1995,30,"Short,Western"
tt0059325,movie,Born in '45,Jahrgang 45,0,1990,100,"Drama,Romance"
tt0059900,movie,"Wenn du groß bist, lieber Adam","Wenn du groß bist, lieber Adam",0,1990,78,"Drama,Fantasy"
tt0065188,movie,"Vojtech, receny sirotek","Vojtech, receny sirotek",0,1990,80,Drama


### Filtrar (coincidencia de texto)

Es posible seleccionar aquellas filas que contienen alguna coincidencia con un texto y no solo aquellas con las que coincide el texto completo.

De base se puede hacer con la función `grepl()`. Esta función sirve para identificar aquellas instancias que cumplen un determinado patrón.

**Ejemplo:** Seleccionar las filas que contengan en *genres* Drama, sin importar si salen más géneros.

In [25]:
df_filtrado <- df[grepl('Drama', df$genres),]
head(df_filtrado)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
1,tt0015724,movie,Dama de noche,Dama de noche,0,1993,102,"Drama,Mystery,Romance"
4,tt0059325,movie,Born in '45,Jahrgang 45,0,1990,100,"Drama,Romance"
5,tt0059900,movie,"Wenn du groß bist, lieber Adam","Wenn du groß bist, lieber Adam",0,1990,78,"Drama,Fantasy"
6,tt0065188,movie,"Vojtech, receny sirotek","Vojtech, receny sirotek",0,1990,80,Drama
8,tt0068494,movie,Domo Arigato,Domo Arigato,0,1990,91,Drama
9,tt0075259,movie,Spy Story,Spy Story,0,1990,103,"Action,Drama,Mystery"


En lo referido a `dplyr`, podemos usar también la función `grepl()` a través de `filter()`.

**Ejemplo:** Seleccionar las filas que contengan en *genres* Drama, sin importar si salen más géneros con `dplyr`.

In [26]:
df_filtrado <- filter(df, grepl('Drama', genres))
head(df_filtrado)

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
tt0015724,movie,Dama de noche,Dama de noche,0,1993,102,"Drama,Mystery,Romance"
tt0059325,movie,Born in '45,Jahrgang 45,0,1990,100,"Drama,Romance"
tt0059900,movie,"Wenn du groß bist, lieber Adam","Wenn du groß bist, lieber Adam",0,1990,78,"Drama,Fantasy"
tt0065188,movie,"Vojtech, receny sirotek","Vojtech, receny sirotek",0,1990,80,Drama
tt0068494,movie,Domo Arigato,Domo Arigato,0,1990,91,Drama
tt0075259,movie,Spy Story,Spy Story,0,1990,103,"Action,Drama,Mystery"


Una opción interesante y que permite `grepl()` es el uso de expresiones regulares.

**Ejemplo:** Seleccionar las filas que contengan en *primaryTitle* un número.

In [27]:
df_filtrado <- df[grepl('[0-9]', df$primaryTitle),]
head(df_filtrado)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
4,tt0059325,movie,Born in '45,Jahrgang 45,0,1990,100,"Drama,Romance"
51,tt0093210,movie,La hora 24,La hora 24,0,1990,98,"Drama,Fantasy"
113,tt0096749,movie,Abatjour 2,Abatjour 2,1,1990,100,"Crime,Mystery,Romance"
128,tt0096845,movie,Atlantic Rhapsody - 52 myndir úr Tórshavn,Atlantic Rhapsody - 52 myndir úr Tórshavn,0,1990,80,"Comedy,Drama"
144,tt0096975,short,Bravo Papa 2040,Bravo Papa 2040,0,1992,6,Short
202,tt0097592,tvMovie,The Iron Butterfly 2,Dak ging 90 II - Zi mong meng tin ngaai,0,1990,NA,"Action,Romance"


**Ejemplo:** Seleccionar las filas que contengan en *primaryTitle* un número con `dplyr`.

In [28]:
df_filtrado <- filter(df, grepl('[0-9]', primaryTitle))
head(df_filtrado)

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
tt0059325,movie,Born in '45,Jahrgang 45,0,1990,100,"Drama,Romance"
tt0093210,movie,La hora 24,La hora 24,0,1990,98,"Drama,Fantasy"
tt0096749,movie,Abatjour 2,Abatjour 2,1,1990,100,"Crime,Mystery,Romance"
tt0096845,movie,Atlantic Rhapsody - 52 myndir úr Tórshavn,Atlantic Rhapsody - 52 myndir úr Tórshavn,0,1990,80,"Comedy,Drama"
tt0096975,short,Bravo Papa 2040,Bravo Papa 2040,0,1992,6,Short
tt0097592,tvMovie,The Iron Butterfly 2,Dak ging 90 II - Zi mong meng tin ngaai,0,1990,NA,"Action,Romance"


## Concatenación de operaciones

Podemos realizar simultáneamente una selección de variables y casos. La opción básica no requiere de mayores explicaciones.

**Ejemplo:** Seleccionar las filas que contengan en *genres* Comedy y las columnas *primaryTitle* y *startYear*.

In [29]:
df_filtrado <- df[grepl('Comedy', df$genres), c('primaryTitle', 'startYear')]
head(df_filtrado)

,primaryTitle,startYear
13,Me and the Kid,1993
23,The Buddhist Spell,1993
28,Tian di xuan men,1991
31,Man(n) sucht Frau,1995
35,Halfaouine: Boy of the Terraces,1990
40,The Scarlet Scorpion,1990


Sin embargo, como ya adelantamos, para concatenar operaciones en `dplyr` tenemos el operador pipe (%\>%). Un aspecto importante en su uso es que en primer lugar indicamos el *data.frame* y tras ello vamos encadenando funciones sin indicar en estas la variable.

**Ejemplo:** Seleccionar las filas que contengan en *genres* Comedy y las columnas *primaryTitle* y *startYear* con `dplyr`.

In [30]:
df_filtrado <- df %>%
  filter(grepl('Comedy', genres)) %>%
  select(primaryTitle, startYear)
head(df_filtrado)

primaryTitle,startYear
Me and the Kid,1993
The Buddhist Spell,1993
Tian di xuan men,1991
Man(n) sucht Frau,1995
Halfaouine: Boy of the Terraces,1990
The Scarlet Scorpion,1990


## Cruzando *data.frames*

Esta es una de las operaciones clave y que en R se puede llevar a cabo de manera muy rápida y sencilla. Al cruzar diferentes datasets tenemos cuatro opciones básicas:

-   **Inner** - Se mantienen las instancias comunes, combinándose las variables de ambos datasets.
-   **Left** - Se mantienen las instancias comunes y todas las demás del dataset que se mantiene a la izquierda, combinándose las variables de ambos datasets. En el caso de las instancias no comunes, los valores de las variables nuevas aparecen como nulos.
-   **Right** - Se mantienen las instancias comunes y todas las demás del dataset que se mantiene a la derecha, combinándose las variables de ambos datasets. En el caso de las instancias no comunes, los valores de las variables nuevas aparecen como nulos.
-   **Full** - Se mantienen las instancias comunes y todas las demás, combinándose las variables de ambos datasets. En el caso de las instancias no comunes, los valores de las variables nuevas aparecen como nulos.

![Fuente: https://www.hostingplus.cl/blog/tipos-de-join-en-sql-cuales-son-los-principales/](images/joins.jpg)

Para este caso vamos a importar un nuevo dataset para cruzarlo con el ya presente. Vamos a importar los datos de votaciones.

In [31]:
df_notas <- read.delim(file = 'data/title_ratings.tsv',
                       quote = '', # no tengo calificadores
                       na.strings = 'null', # caracteres nulos
                       encoding = 'UTF-8') # codificación

df_akas <- read.delim(file = 'data/title_akas.tsv',
                      quote = '', # no tengo calificadores
                      na.strings = 'null', # caracteres nulos
                      encoding = 'UTF-8') # codificación

En lugar de usar la opción base de R vamos a optar directamente por el paquete `dplyr`, ya que está más optimizado al respecto. De este modo contamos con las funciones `inner_join()`, `left_join()`, `right_join()` y `full_join()`. En todos ellos la forma de trabajar es la misma, se indica el dataset de la izquierda, el de la derecha y la variable en común para cruzarlos.

**Ejemplo:** Combinar los datos de películas con los datos de sus respectivas notas dejando solo las películas que tienen notas con `dplyr`.

In [32]:
df_filtrado <- inner_join(x=df, y=df_notas, by='tconst')
head(df_filtrado)

Warning message:
“Column `tconst` joining character vector and factor, coercing into character vector”

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
tt0015724,movie,Dama de noche,Dama de noche,0,1993,102,"Drama,Mystery,Romance",6.1,26
tt0038086,movie,Shiva und die Galgenblume,Shiva und die Galgenblume,0,1993,NA,Thriller,7.0,26
tt0040844,short,Crossroads of Laredo,Crossroads of Laredo,0,1995,30,"Short,Western",3.3,186
tt0059325,movie,Born in '45,Jahrgang 45,0,1990,100,"Drama,Romance",6.4,249
tt0059900,movie,"Wenn du groß bist, lieber Adam","Wenn du groß bist, lieber Adam",0,1990,78,"Drama,Fantasy",6.6,37
tt0065188,movie,"Vojtech, receny sirotek","Vojtech, receny sirotek",0,1990,80,Drama,6.5,36


**Ejemplo:** Combinar los datos de películas con los datos de sus respectivas notas dejando tanto las películas que tienen notas como las que no con `dplyr`.

In [33]:
df_filtrado <- left_join(x=df, y=df_notas, by='tconst')
df_filtrado <- full_join(x=df, y=df_notas, by='tconst') # no hay notas sin películas
head(df_filtrado)

Warning message:
“Column `tconst` joining character vector and factor, coercing into character vector”Warning message:
“Column `tconst` joining character vector and factor, coercing into character vector”

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
tt0015724,movie,Dama de noche,Dama de noche,0,1993,102,"Drama,Mystery,Romance",6.1,26
tt0038086,movie,Shiva und die Galgenblume,Shiva und die Galgenblume,0,1993,NA,Thriller,7.0,26
tt0040844,short,Crossroads of Laredo,Crossroads of Laredo,0,1995,30,"Short,Western",3.3,186
tt0059325,movie,Born in '45,Jahrgang 45,0,1990,100,"Drama,Romance",6.4,249
tt0059900,movie,"Wenn du groß bist, lieber Adam","Wenn du groß bist, lieber Adam",0,1990,78,"Drama,Fantasy",6.6,37
tt0065188,movie,"Vojtech, receny sirotek","Vojtech, receny sirotek",0,1990,80,Drama,6.5,36


Puede darse el caso también de que la variable que utilizamos para hacer el join se llame de forma diferente en ambos datasets. Entre las alternativas posibles para sortear este contratiempo la más rápida pasa por indicarle esta asociación a la función de join que estemos usando.

**Ejemplo:** Combinar los datos de películas con los datos de sus títulos en otros idiomas dejando solo las películas con títulos traducidos con `dplyr`.

In [34]:
df_filtrado <- inner_join(x=df, y=df_akas, by=c('tconst'='titleId'))
head(df_filtrado)

Warning message:
“Column `tconst`/`titleId` joining character vector and factor, coercing into character vector”

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,ordering,title,region,language,types,attributes,isOriginalTitle
tt0015724,movie,Dama de noche,Dama de noche,0,1993,102,"Drama,Mystery,Romance",1,Dama de noche,MX,NA,imdbDisplay,NA,0
tt0015724,movie,Dama de noche,Dama de noche,0,1993,102,"Drama,Mystery,Romance",2,Dama de noche,NA,NA,original,NA,1
tt0015724,movie,Dama de noche,Dama de noche,0,1993,102,"Drama,Mystery,Romance",3,Éjszakai gyilkosság,HU,NA,imdbDisplay,NA,0
tt0015724,movie,Dama de noche,Dama de noche,0,1993,102,"Drama,Mystery,Romance",4,Lady of the Night,NA,NA,NA,NA,0
tt0038086,movie,Shiva und die Galgenblume,Shiva und die Galgenblume,0,1993,NA,Thriller,1,Shiva und die Galgenblume,DE,NA,imdbDisplay,NA,0
tt0038086,movie,Shiva und die Galgenblume,Shiva und die Galgenblume,0,1993,NA,Thriller,2,Shiva und die Galgenblume,NA,NA,original,NA,1


## Agrupando datos

Otra de las operaciones más populares es la de agregar datos. Tomando una o varias columnas podemos agregar las variables asociadas a ellas y aplicar alguna operación a estos datos agregados (recuento, suma, promedio...). Nuevamente, vamos a centrarnos en la opción que ofrece `dplyr` a través de `group_by()`. No obstante, en este caso también necesitamos usar la función `summarise()`, pues con la primera indicamos cómo agrupamos y con esta última la operación realizada sobre el resto de variables.

**Ejemplo:** Calcular el año de publicación (*startYear*) promedio de cada tipología (*titleType*) en `dplyr`.

In [35]:
df_filtrado <- df %>%
  group_by(titleType) %>%
  summarise(startYear_avg = mean(startYear))
head(df_filtrado)

titleType,startYear_avg
movie,1994.561
short,1995.407
tvMovie,1994.888
tvShort,1994.530
videoGame,1994.587


**Ejemplo:** Calcular el número de obras que se han publicado anualmente (*startYear*) por tipología (*titleType*) en `dplyr`.

In [36]:
df_filtrado <- df %>%
  group_by(titleType, startYear) %>%
  summarise(total = n())
head(df_filtrado)

titleType,startYear,total
movie,1990,4842
movie,1991,4393
movie,1992,4321
movie,1993,4156
movie,1994,4125
movie,1995,4303


## Algunos casos prácticos

A modo de ejercicios para poner a prueba lo aprendido hasta ahora y captar mejor el potencial de estas opciones, vamos a llevar a cabo diferentes casos prácticos. Todos ellos se van a realizar en `dplyr`.

**Caso 1:** Obtener el listado con la película más votada de cada año.

In [37]:
df_filtrado <- df %>%
  inner_join(df_notas, by='tconst') %>%
  filter(titleType == 'movie') %>%
  group_by(startYear) %>%
  filter(numVotes == max(numVotes)) %>%
  select(startYear, primaryTitle, numVotes)
df_filtrado

Warning message:
“Column `tconst` joining character vector and factor, coercing into character vector”

startYear,primaryTitle,numVotes
1990,Goodfellas,1148796
1991,The Silence of the Lambs,1417722
1992,Reservoir Dogs,1010002
1993,Schindler's List,1343514
1994,The Shawshank Redemption,2651251
1995,Se7en,1632691
1996,Trainspotting,684236
1997,Titanic,1162341
1998,Saving Private Ryan,1378178
1999,Fight Club,2096443


**Caso 2:** Obtener el listado de las 20 películas con mayor puntuación promedio, habiéndolas votado al menos 50000 usuarios, junto al género/s.

In [38]:
df_filtrado <- df %>%
  inner_join(df_notas, by='tconst') %>%
  filter(titleType == 'movie' & numVotes >= 50000) %>%
  arrange(desc(averageRating)) %>%
  slice(1:20) %>%
  select(primaryTitle, genres, averageRating)
df_filtrado

Warning message:
“Column `tconst` joining character vector and factor, coercing into character vector”

primaryTitle,genres,averageRating
The Shawshank Redemption,Drama,9.3
Schindler's List,"Biography,Drama,History",9.0
Pulp Fiction,"Crime,Drama",8.9
Forrest Gump,"Drama,Romance",8.8
Fight Club,Drama,8.8
Goodfellas,"Biography,Crime,Drama",8.7
The Matrix,"Action,Sci-Fi",8.7
The Silence of the Lambs,"Crime,Drama,Thriller",8.6
Terminator 2: Judgment Day,"Action,Sci-Fi",8.6
Se7en,"Crime,Drama,Mystery",8.6


**Caso 3:** El listado anterior pero con los títulos en Español.

In [39]:
df_filtrado <- df %>%
  inner_join(df_notas, by='tconst') %>%
  filter(titleType == 'movie' & numVotes >= 50000) %>%
  arrange(desc(averageRating)) %>%
  slice(1:20) %>%
  inner_join(df_akas, by=c('tconst'='titleId')) %>%
  filter(region=='ES' & is.na(language) & is.na(attributes)) %>%
  select(title, genres, averageRating)
df_filtrado

Warning message:
“Column `tconst` joining character vector and factor, coercing into character vector”Warning message:
“Column `tconst`/`titleId` joining character vector and factor, coercing into character vector”

title,genres,averageRating
Cadena perpetua,Drama,9.3
La lista de Schindler,"Biography,Drama,History",9.0
Pulp Fiction,"Crime,Drama",8.9
Forrest Gump,"Drama,Romance",8.8
El club de la lucha,Drama,8.8
Uno de los nuestros,"Biography,Crime,Drama",8.7
Matrix,"Action,Sci-Fi",8.7
El silencio de los corderos,"Crime,Drama,Thriller",8.6
Terminator 2: El juicio final,"Action,Sci-Fi",8.6
Seven,"Crime,Drama,Mystery",8.6


**Caso 4:** Calcular el total anual de películas de comedia valoradas por un mínimo de 1000 personas, y su valoración y duración promedio.

In [40]:
df_filtrado <- df %>%
  inner_join(df_notas, by='tconst') %>%
  filter(grepl('Comedy', genres) & titleType == 'movie' & numVotes >= 1000) %>%
  group_by(startYear)  %>%
  summarise(movies = n(), averageRating_avg = mean(averageRating), runtimeMinutes = mean(runtimeMinutes, na.rm=TRUE))
df_filtrado

Warning message:
“Column `tconst` joining character vector and factor, coercing into character vector”

startYear,movies,averageRating_avg,runtimeMinutes
1990,146,6.154795,99.71918
1991,155,6.289677,101.26452
1992,139,6.211511,102.83453
1993,150,6.281333,101.90667
1994,185,6.052973,103.01622
1995,157,6.237580,101.42675
1996,185,6.217297,102.23784
1997,199,6.103518,102.14573
1998,212,6.353774,104.88208
1999,230,6.191739,102.06987


**Caso 5:** Obtener el listado de los 10 videojuegos publicados entre 1995 y 1997 con mayor puntuación promedio, habiéndolos votado al menos 5000 usuarios, junto al año.

In [41]:
df_filtrado <- df %>%
  inner_join(df_notas, by='tconst') %>%
  filter(titleType == 'videoGame' & numVotes >= 5000) %>%
  arrange(desc(averageRating)) %>%
  slice(1:10) %>%
  select(primaryTitle, startYear, averageRating)
df_filtrado

Warning message:
“Column `tconst` joining character vector and factor, coercing into character vector”

primaryTitle,startYear,averageRating
Metal Gear Solid,1998,9.6
The Legend of Zelda: Ocarina of Time,1998,9.6
Final Fantasy VII,1997,9.5
StarCraft,1998,9.2
Half-Life,1998,9.2
Resident Evil 2,1998,9.1
Super Mario 64,1996,9.1
GoldenEye 007,1997,9.1
Silent Hill,1999,9.0
Resident Evil,1996,8.9


# Exportar

En última instancia, llega el turno de exportar los datos. Nuevamente contamos con la opción base y la ofrecida por el paquete `data.table`.

De base tenemos la función `write.table()`, que es la general y exporta por defecto en formato separado por tabuladores, y sus versiones derivadas que los hacen en separado por comas `write.csv()` y `write.csv2()`. Se trata de un proceso muy parecido al de lectura, donde debemos dejar claro el formato del archivo para garantizar que este sea después utilizable.

**Ejemplo:** Exportar el *data.frame* del último caso práctico separado por tabuladores.

In [42]:
write.table(df_filtrado, # data.frame a exportar
            'results/archivo_exportado.tsv', # ruta y nombre de archivo
            quote = FALSE, # en este caso no quiero ya que no necesito
            fileEncoding = 'UTF-8', # codificación
            row.names = FALSE) # omito los nombres/indices de filas

**Ejemplo:** Exportar el *data.frame* del último caso práctico separado por comas.

In [43]:
write.csv(df_filtrado, # data.frame a exportar
          'results/archivo_exportado.csv', # ruta y nombre de archivo
          fileEncoding = 'UTF-8', # codificación
          row.names = FALSE) # omito los nombres/indices de filas

Con `data.table` el proceso se realiza con la función `fwrite()` y es igual de sencillo. Por defecto utiliza el calificador cuando detecta que es necesario, y usa de separador las comas.

In [44]:
fwrite(df_filtrado, # data.frame a exportar
       'results/archivo_exportado.csv') # ruta y nombre de archivo